## Step 5: Upward Sweep

In [1]:
import numpy
from treecode_helper import Particle, Cell, build_tree

In the previous notebook, we define a cell class, and build the octree by adding particles into cells and splitting cells recursively. So far, we covered how to use multipole expansion to accelerate the potential evaluation, and how to construct the hierarchical tree to group the particles in different cells. In this notebook, we will exploit the advantage of tree structure to calculate the multipoles for cells. Before we start, we create a python script "treecode.py" in the same folder which contains the class definitions and functions we discussed before. We import this script as a module. Now we can reuse the code to generate the tree (the list of cells) as follows. 

In [2]:
n = 100          # number of particles
particles = [ Particle(m=1.0/n) for i in range(n) ]

n_crit = 10      # max number of particles in a single cell

In [3]:
root = Cell(n_crit)
root.x, root.y, root.z = 0.5, 0.5, 0.5
root.r = 0.5

In [4]:
cells = build_tree(particles, root, n_crit)

In [5]:
def get_multipole(particles, p, cells, leaves, n_crit):
    """Calculate multipole arrays for all leaf cells under cell p. If leaf number of cell p is equal or bigger than n_crit (non-leaf), traverse down recursively. Otherwise (leaf), calculate the multipole arrays for leaf cell p.
    
    Arguments:
        p: current cell's index
        cells: the list of cells
        leaves: the array of all leaf cells
        n_crit: maximum number of leaves in a single cell
      
    """
    if cells[p].nleaf >= n_crit:
        for c in range(8):
            if cells[p].nchild & (1 << c):
                get_multipole(particles, cells[p].child[c], cells, leaves, n_crit)
    else:
        # loop in leaf particles
        for i in range(cells[p].nleaf):
            l = cells[p].leaf[i]
            dx, dy, dz = cells[p].x-particles[l].x, cells[p].y-particles[l].y, cells[p].z-particles[l].z
            # monopole: 1 term
            cells[p].multipole[0] += particles[l].m
            # dipole: 3 terms
            cells[p].multipole[1:4] += particles[l].m * numpy.array((dx, dy, dz))
            # quadruple: 6 terms
            cells[p].multipole[4:] += particles[l].m/2 * numpy.array((dx**2, dy**2, dz**2,\
                                                            dx*dy, dy*dz, dz*dx))
        leaves.append(p)

In [6]:
leaves = []
get_multipole(particles, 0, cells, leaves, n_crit)

In [7]:
def M2M(p, c, cells):
    """Calculate parent cell p's multipole array based on child cell c's multipoles
    
    Arguments:
        p: parent cell index in cells list
        c: child cell index in cells list
        cells: the list of cells
    """
    dx, dy, dz = cells[p].x-cells[c].x, cells[p].y-cells[c].y, cells[p].z-cells[c].z
    # monopole: 1 term
    cells[p].multipole[0] += cells[c].multipole[0]
    # dipole：3 terms
    cells[p].multipole[1:4] += cells[c].multipole[1:4] + cells[c].multipole[0]*numpy.array((dx, dy, dz))
    # quadruple: 6 terms
    cells[p].multipole[4] += cells[c].multipole[4] + dx * cells[c].multipole[1] \
                                                   + dx * dx * cells[c].multipole[0] / 2
    cells[p].multipole[5] += cells[c].multipole[5] + dy * cells[c].multipole[2] \
                                                   + dy * dy * cells[c].multipole[0] / 2
    cells[p].multipole[6] += cells[c].multipole[6] + dz * cells[c].multipole[3] \
                                                   + dz * dz * cells[c].multipole[0] / 2
    cells[p].multipole[7] += cells[c].multipole[7] + (dx * cells[c].multipole[2] \
                                                   +  dy * cells[c].multipole[1] \
                                                   +  dx * dy * cells[c].multipole[0]) / 2
    cells[p].multipole[8] += cells[c].multipole[8] + (dy * cells[c].multipole[3] \
                                                   +  dz * cells[c].multipole[2] \
                                                   +  dy * dz * cells[c].multipole[0]) / 2
    cells[p].multipole[9] += cells[c].multipole[9] + (dz * cells[c].multipole[1] \
                                                   +  dx * cells[c].multipole[3] \
                                                   +  dz * dx * cells[c].multipole[0]) / 2

In [8]:
def upward_sweep(cells):
    """Traverse from leaves to root, in order to calculate multipoles of all the cells.
    
    Arguments:
        cells: the list of cells
    
    """
    for c in range(len(cells)-1, 0, -1):
        p = cells[c].parent
        M2M(p, c, cells)

In [13]:
upward_sweep(cells)

##### Reference

1. R. Yokota, 12 Steps to a Fast Multipole Method on GPUs, Pan-American Advanced Studies Institute, Valparaiso, Chile, 3-14 January, 2011.
2. Raykar, V. C., "[A short primer on the fast multipole method: FMM tutorial](http://www.umiacs.umd.edu/labs/cvl/pirl/vikas/publications/FMM_tutorial.pdf),", University of Maryland, College Park, Apr. 8, 2006.

In [9]:
from IPython.core.display import HTML
def css_styling():
    styles = open('./style/fmmstyle.css', 'r').read()
    return HTML(styles)
css_styling()